In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from konlpy.tag import Okt, Kkma, Komoran
from collections import Counter
import pickle

In [2]:
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'Malgun Gothic'
fontpath = 'C:/Windows/Fonts/malgun.ttf'

In [3]:
path = 'workbookdata/입찰결과물품_df.csv'
입찰결과물품df = pd.read_csv(path, encoding='cp949')
결과_입찰공고명 = 입찰결과물품df.입찰공고명

In [35]:
# 의료 관련 키워드 (이전 작업에서 사용된 것으로 가정)
medical_keywords = ['병원', '환자', '의무', '외상', '안과', '자극', '신경', '의학', \
                 '이비인후과', '치과', '붕대', '외과', '수술', '의료', '병리', '예방약', \
                 '방사선장비', '의약품', '치료', '안경제작', '위생']

# 의료 키워드로 필터링된 result_df (이전 작업 결과로 가정)
h_pattern = '|'.join(medical_keywords)
health_df = 입찰결과물품df[입찰결과물품df['입찰공고명'].str.contains(h_pattern, na=False, regex=True)]

# 입찰결과물품df에서 result_df를 제외
remaining_df = 입찰결과물품df[~입찰결과물품df.index.isin(health_df.index)]

# 필요한 컬럼만 선택
remaining_df = remaining_df[['입찰공고명', '공고기관명']]

# 결과 출력
print(f"남은 행 수: {len(remaining_df)}")
print(remaining_df)

남은 행 수: 57848
                                 입찰공고명    공고기관명
0      00부대 물절약 투자대행사업(WASCO  15-A048)  국군재정관리단
1                          155밀리 모의신관류    방위사업청
2                          155밀리 모의신관류    방위사업청
3                          155밀리 모의신관류    방위사업청
4                         스펙트럼용 분석기 구매  국군재정관리단
...                                ...      ...
61427    심해잠수체계(PTC  Wet Bell) 유지보수 정비  해군군수사령부
61428     24-B-대형 에어컨 등 5품목 구매 납품 및 설치  제9196부대
61429         (246361-D) 소형 탑승식 청소차 구매  국군재정관리단
61430             (246343-D) VR HMD 구매  국군재정관리단
61431                24-M-공업용 헬륨가스 구매   국군재정관리단

[57848 rows x 2 columns]


In [42]:
### DF 로 찾기
const_keywords = ['목재류','건축용', '아파트', '숙소', '제조설치', '철재', '철근', '철선', \
                  '페인트', '시멘트', '보일러', '콘크리트', '아스팔트', '벽돌', '타일', '생활관', \
                  '토목', '강관']
# ['콘크리트', '철근', '시멘트', '목재', '강관', '아스팔트', '벽돌', '타일', '건축', '토목']
remove_keywords = ['병원', '의무', '치료']
cst_pattern = '|'.join(const_keywords)
r_pattern = '|'.join(remove_keywords) if remove_keywords and remove_keywords != [''] else None

# '방사선' 포함 공고명 필터링
const_df = remaining_df[remaining_df['입찰공고명'].str.contains(cst_pattern, na=False, regex=True)]

# 'remove_keywords'가 비어 있지 않으면 제거 필터링 적용
if r_pattern:
    const_df = const_df[~const_df['입찰공고명'].str.contains(r_pattern, na=False, regex=True)]
# 필요한 컬럼만 선택
const_df = const_df[['입찰공고명', '공고기관명']]

# 결과 출력
print(f"건설관련 결과 건수: {len(const_df)}")
print(const_df)

건설관련 결과 건수: 2055
                                입찰공고명       공고기관명
34       13-본-국대-01 00학교이전 공기조화기 제조설치     국군재정관리단
272                  간부숙소 알루미늄창호 제조설치     국군재정관리단
315              영내관사 보일러 및 분배기 구매/설치  공군제11전투비행단
319                계룡대 비상용발전기 제조설치 납품     국군재정관리단
343        16-D-021-철재 장의자 등 5종 제조 납품     국군재정관리단
...                               ...         ...
61267     주-대-1 일산에버나인 아파트 싱크대 제조설치납품   제9보병사단사령부
61305  [24-04]00부대 실내체육관 신축 자동제어 제조설치     제3007부대
61334       (246366-E) 기동형 방송시스템 제조설치     국군재정관리단
61353   24-미-5 미사일방어태세 보강 공사 방호벽 제조설치  공군미사일방어사령부
61394            24-N-0함대 부두 로딩암 제조설치     국군재정관리단

[2055 rows x 2 columns]


In [19]:
### 실험용 DF 로 찾기
test = ['']
remove_keywords = ['목재류','건축용', '철재', '철근', '철선', '페인트', '시멘트', '콘크리트', '아스팔트', '벽돌', '타일']
t_pattern = '|'.join(test)
r_pattern = '|'.join(remove_keywords) if remove_keywords and remove_keywords != [''] else None

# '방사선' 포함 공고명 필터링
test_df = 입찰결과물품df[입찰결과물품df['입찰공고명'].str.contains(t_pattern, na=False, regex=True)]

# 'remove_keywords'가 비어 있지 않으면 제거 필터링 적용
if r_pattern:
    test_df = test_df[~test_df['입찰공고명'].str.contains(r_pattern, na=False, regex=True)]
# 필요한 컬럼만 선택
test_df = test_df[['입찰공고명', '공고기관명']]

# 결과 출력
print(f"건설관련 결과 건수: {len(test_df)}")
test_df.head(30)

건설관련 결과 건수: 128


,입찰공고명,공고기관명
4666,00부대 병영생활관 신축 플라스틱 창호 제조납품,육군 제1군단
6514,112공병 생활관 개수 진공온수보일러 제조설치납품,제2군단
6515,112공병 병영생활관 개수 PW창호 제조납품,제2군단
6560,46 영산강소초 GOP 병영생활관,제1862부대
7281,000사업 병영생활관 통합배선반 제조설치,국군재정관리단
7282,000사업 병영생활관 합성수지제창호 제조,국군재정관리단
8028,00지역 병영생활관 신축 PHC파일 구매,제2136부대
8682,00지역 병영생활관 신축 PHC파일 구매,제2136부대
9023,병영생활관 신축공사 관급자재 배전반,제5861부대
9024,병영생활관 신축공사 관급자재 분전반,제5861부대


In [9]:
입찰결과공고명df = pd.DataFrame(결과_입찰공고명)
# 입찰결과공고명df

In [18]:
folder_path = 'workbookdata'

# CSV 파일로 저장
output_path = os.path.join(folder_path, 'ResultConstDF.csv')
const_df.to_csv(output_path, encoding='cp949')

In [ ]:
메모리 비우기
binList = []
입찰공고token = []
df = pd.DataFrame(binList)